### LIBRARIES

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import string
import random


### DATA LOADING AND VISUALISATION

In [9]:
import pandas as pd
from collections import Counter
# Load the CSV file
df = pd.read_csv("/kaggle/input/info-sec-a2/Balanced_dataset.csv")

# Display basic information
print(df.head())  # Show first 5 rows
print(df.info())  # Get data types and non-null values


                                                 url    type scheme  \
0    https://ethnicelebs.com/eddie-anderson-comedian  benign  https   
1  https://closinglogos.com/page/Sony+Pictures+Ho...  benign  https   
2  https://terezowens.com/golfer-greg-norman-tryi...  benign  https   
3                        https://americannortel.com/  benign  https   
4                               https://welding.org/  benign  https   

               domain subdomain top_level_domain  \
0     ethnicelebs.com      none              com   
1    closinglogos.com      none              com   
2      terezowens.com      none              com   
3  americannortel.com      none              com   
4         welding.org      none              org   

                                                path  path_length  \
0                           /eddie-anderson-comedian           24   
1  /page/Sony+Pictures+Home+Entertainment+Warning...           53   
2  /golfer-greg-norman-trying-to-unload-55-millio... 

In [10]:
df

,url,type,scheme,domain,subdomain,top_level_domain,path,path_length,num_path_segments,query_params,num_query_params,has_https,file_extension,has_fragment,has_special_chars_in_path,has_port,port_number,is_ip_address
0,https://ethnicelebs.com/eddie-anderson-comedian,benign,https,ethnicelebs.com,none,com,/eddie-anderson-comedian,24,1,0,0,1,none,0,0,0,0,0
1,https://closinglogos.com/page/Sony+Pictures+Ho...,benign,https,closinglogos.com,none,com,/page/Sony+Pictures+Home+Entertainment+Warning...,53,2,0,0,1,none,0,0,0,0,0
2,https://terezowens.com/golfer-greg-norman-tryi...,benign,https,terezowens.com,none,com,/golfer-greg-norman-trying-to-unload-55-millio...,54,1,0,0,1,none,0,0,0,0,0
3,https://americannortel.com/,benign,https,americannortel.com,none,com,/,1,0,0,0,1,none,0,0,0,0,0
4,https://welding.org/,benign,https,welding.org,none,org,/,1,0,0,0,1,none,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59600,http://archives.li.man.ac.uk/ead/search/?opera...,spam,http,archives.li.man.ac.uk,archives,uk,/ead/search/,12,2,6,6,0,none,0,0,0,0,0
59601,http://amber.ch.ic.ac.uk/archive/all/10253.html,spam,http,amber.ch.ic.ac.uk,amber,uk,/archive/all/10253.html,23,3,0,0,0,html,0,0,0,0,0
59602,http://archive.thisischeshire.co.uk/2000/1/28/...,spam,http,archive.thisischeshire.co.uk,archive,uk,/2000/1/28/225238.html,22,4,0,0,0,html,0,0,0,0,0
59603,http://ads.guardian.co.uk/click.ng/Params.rich...,spam,http,ads.guardian.co.uk,ads,uk,/click.ng/Params.richmedia=yes&location=middle...,128,2,0,0,0,richmedia=yes&location=middle&spacedesc=06&sit...,0,1,0,0,0


In [11]:
df.columns

Index(['url', 'type', 'scheme', 'domain', 'subdomain', 'top_level_domain',
       'path', 'path_length', 'num_path_segments', 'query_params',
       'num_query_params', 'has_https', 'file_extension', 'has_fragment',
       'has_special_chars_in_path', 'has_port', 'port_number',
       'is_ip_address'],
      dtype='object')

In [12]:
df['type'].value_counts()

type
benign        11921
defacement    11921
phishing      11921
malware       11921
spam          11921
Name: count, dtype: int64

In [13]:
df[df.columns][:5]

,url,type,scheme,domain,subdomain,top_level_domain,path,path_length,num_path_segments,query_params,num_query_params,has_https,file_extension,has_fragment,has_special_chars_in_path,has_port,port_number,is_ip_address
0,https://ethnicelebs.com/eddie-anderson-comedian,benign,https,ethnicelebs.com,none,com,/eddie-anderson-comedian,24,1,0,0,1,none,0,0,0,0,0
1,https://closinglogos.com/page/Sony+Pictures+Ho...,benign,https,closinglogos.com,none,com,/page/Sony+Pictures+Home+Entertainment+Warning...,53,2,0,0,1,none,0,0,0,0,0
2,https://terezowens.com/golfer-greg-norman-tryi...,benign,https,terezowens.com,none,com,/golfer-greg-norman-trying-to-unload-55-millio...,54,1,0,0,1,none,0,0,0,0,0
3,https://americannortel.com/,benign,https,americannortel.com,none,com,/,1,0,0,0,1,none,0,0,0,0,0
4,https://welding.org/,benign,https,welding.org,none,org,/,1,0,0,0,1,none,0,0,0,0,0


In [14]:
import pandas as pd

# Load dataset
df2 = pd.read_csv("/kaggle/input/info-sec-a2/Balanced_dataset.csv")

df2 = df2.fillna("")
df2["text"] = df2["url"].astype(str) + " " + df2["scheme"].astype(str) + " " + df2["domain"].astype(str) + " " + \
              df2["subdomain"].astype(str) + " " + df2["top_level_domain"].astype(str) + " " + \
              df2["path"].astype(str) + " " + df2["file_extension"].astype(str)

# Check for unexpected non-string types
assert df2["text"].apply(lambda x: isinstance(x, str)).all(), "Error: Some values are not strings!"
# Check data

print(df2[["text", "type"]].head())


                                                text    type
0  https://ethnicelebs.com/eddie-anderson-comedia...  benign
1  https://closinglogos.com/page/Sony+Pictures+Ho...  benign
2  https://terezowens.com/golfer-greg-norman-tryi...  benign
3  https://americannortel.com/ https americannort...  benign
4  https://welding.org/ https welding.org none or...  benign


In [59]:
CLASS_MAPPING = {"benign": 0, "defacement": 1, "phishing": 2, "malware": 3, "spam": 4}
df2["label"] = df2["type"].map(CLASS_MAPPING)  # Convert text labels to numbers
data=df2[["url", "label"]]

### TOKENSIZER AND DATA LOADER

In [ ]:
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

def char_tokenizer(url, max_length=50):
    url = url.lower()[:max_length].ljust(max_length, " ")  # Pad or truncate
    return [ord(char) if ord(char) < 128 else 0 for char in url]  # Convert to ASCII IDs

class URLDataset(Dataset):
    def __init__(self, urls, labels, max_length=50):
        self.urls = [char_tokenizer(url, max_length) for url in urls]
        self.labels = labels.astype(int)

    def __len__(self):
        return len(self.urls)

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.urls[idx], dtype=torch.long)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return {"input_ids": input_ids, "label": label}

# Create Dataset & DataLoader
train_dataset = URLDataset(train_data["url"].values, train_data["label"].values)
test_dataset = URLDataset(test_data["url"].values, test_data["label"].values)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


### LLM MODEL CLASS

In [ ]:
class CustomLLM(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_dim, num_layers, num_classes, max_length):
        super(CustomLLM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.position_embeddings = nn.Parameter(torch.randn(1, max_length, embed_dim))

        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=ff_dim, activation="gelu", batch_first=True)
            for _ in range(num_layers)
        ])

        self.norm = nn.LayerNorm(embed_dim)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, input_ids):
        x = self.embedding(input_ids) + self.position_embeddings
        for layer in self.transformer_layers:
            x = layer(x)
        x = self.norm(x[:, 0, :])  # Use first token representation
        logits = self.fc(x)
        return logits

### TRAIN AND EVALUTION FUNCTIONS

In [ ]:

def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        loop = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}")
        total_loss = 0
        correct, total = 0, 0

        for batch in loop:
            input_ids = batch["input_ids"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            total_loss += loss.item()
            loop.set_postfix(loss=total_loss / (total + 1), acc=correct / total)

def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0,0
    total_loss = 0

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

    print(f"Validation Loss: {total_loss / len(test_loader):.4f}")
    print(f"Validation Accuracy: {correct / total:.4f}")



### MODEL INITIALISATION

In [62]:


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VOCAB_SIZE = 128  # ASCII characters
EMBED_DIM = 256
NUM_HEADS = 8
FF_DIM = 512
NUM_LAYERS = 6
MAX_LENGTH = 50
NUM_CLASSES = 5  # Number of URL categories
BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 3e-4

model = CustomLLM(VOCAB_SIZE, EMBED_DIM, NUM_HEADS, FF_DIM, NUM_LAYERS, NUM_CLASSES, MAX_LENGTH).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)




### MODEL TRAINING

In [63]:
train_model(model, train_loader, criterion, optimizer, epochs=EPOCHS)



Epoch 5/5: 100%|██████████| 1491/1491 [00:25<00:00, 58.54it/s, acc=0.926, loss=0.00647]


### MODEL TESTING

In [67]:
evaluate_model(model, test_loader)

Evaluating: 100%|██████████| 373/373 [00:01<00:00, 262.96it/s]

Validation Loss: 0.2298
Validation Accuracy: 0.9189
